c:\learning\childCareDessert\childcare\scripts


state_name state   pop04  capacity_total_cc  capacity_total_fdh  \
stFips Year                                                                   
1      2000    Alabama    AL    3519               54.0                 0.0   
       2001    Alabama    AL    3471               54.0                 0.0   
       2002    Alabama    AL    3537               54.0                 0.0   
       2003    Alabama    AL   46628               54.0                 0.0   
       2004    Alabama    AL   46477               54.0                 0.0   
...                ...   ...     ...                ...                 ...   
55     2019  Wisconsin    WI  333184           144373.0              2635.0   
       2020  Wisconsin    WI  331066           142533.0              2167.0   
       2021  Wisconsin    WI  329324           143400.0              2132.0   
       2022  Wisconsin    WI  321594           143100.0              2058.0   
       2023  Wisconsin    WI  321594           137220.0              1763.0   

             capacity_total_other  
stFips Year                        
1      2000                   0.0  
       2001                   0.0  
       2002                   0.0  
       2003                   0.0  
       2004                   0.0  
...                           ...  
55     2019                   0.0  
       2020                   0.0  
       2021                   0.0  
       2022                   0.0  
       2023                   0.0  

[777 rows x 6 columns]

In [11]:
df_raw = df.groupby(by=['State','Year']).sum()[['Other_CC','Total_CC','FDH']].reset_index()

In [12]:
df_single_2020 = df_raw[df_raw['Year']==2020]

In [13]:
df_all_years = pd.DataFrame()
for i in range(2000,2021):
    df_single_2020['Year'] = i
    df_all_years = pd.concat([df_all_years,df_single_2020])
    
df_all_years = df_all_years.reindex()

C:\Users\ese95\AppData\Local\Temp\ipykernel_32740\2177124246.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_single_2020['Year'] = i


In [14]:
df_all_years

,State,Year,Other_CC,Total_CC,FDH
1,AK,2000,41,60,41
3,AL,2000,431,20185,431
5,AR,2000,412,3090,412
7,AZ,2000,130,29535,130
9,CA,2000,494,285532,494
...,...,...,...,...,...
91,VT,2020,80,186,80
93,WA,2020,271,13211,271
95,WI,2020,427,3933,427
97,WV,2020,212,1171,212


In [73]:
import pandas as pd
## load raw data 
raw_data = pd.read_csv('../data/cc_cty_panel_.csv')
# raw_data_cols = raw_data.columns
raw_df = raw_data.fillna(0)


## mapping files
stfips = pd.read_csv('../data/state_fips.csv')
countyfips = pd.read_csv('../data/counties_fips.csv')


In [80]:
import pandas as pd
import os
print(os.getcwd())
## load raw data 
raw_data = pd.read_csv('../data/cc_cty_panel_.csv')
# raw_data_cols = raw_data.columns
raw_df = raw_data.fillna(0)


## mapping files
stfips = pd.read_csv('../data/state_fips.csv')
countyfips = pd.read_csv('../data/counties_fips.csv')

 

county_col_mapper = {
    'Year': 'Year',
    'center_count_cc': 'Total_CC',
    'center_count_fdh': 'FDH',
    'center_count_other': 'Other_CC',
    'state': 'State',
    'county_name':'County',
    'cty_fips':'cty_fips'
}

trend_col_mapper = {
    'stFips': 'stFips',
    'Year': 'Year',
    'state_name':'state_name',
    'state':'state',
    'pop4': 'pop',
    'center_count_cc': 'Total_CC',
    'center_count_fdh': 'FDH',
    'center_count_other': 'Other_CC'
}
state_col_mapper = {
    'Year': 'Year',
    'center_count_cc': 'Total_CC',
    'center_count_fdh': 'FDH',
    'center_count_other': 'Other_CC',
    'stFips': 'stFips'
}
# state data
state_cols = ['center_count_cc','center_count_fdh','center_count_other']
raw_data_full = pd.merge(left=raw_df,right=stfips,on=['stFips'])
raw_data_full = raw_data_full.merge(countyfips,on=['cty_fips','state'])


state_df = raw_data_full.groupby(by=['stFips','Year']).sum()[state_cols]
state_df.head()


inusefips = stfips[['stFips','state_name','state']]

in_data_state = set(state_df.index.get_level_values(level='stFips'))

min_year = 2000
max_year = 2024


for idx,row in stfips[['stFips','state_name','state']].iterrows():
    prev_pop = 0    
    prev_total = 0
    prev_other = 0
    prev_fdh = 0
    state_name = row['state_name']
    state = row['state']
    state_fip = row['stFips']
    for yr in range(min_year,max_year+1,1):
        pair = (state_fip,yr)
        # print(state_name)
        state_df.loc[(state_fip,yr),'state_name'] = state_name
        state_df.loc[(state_fip,yr),'state'] = state
        if pair not in state_df.index:
            
            state_df.loc[(state_fip,yr),'center_count_cc'] = prev_total
            state_df.loc[(state_fip,yr),'center_count_fdh'] = prev_other
            state_df.loc[(state_fip,yr),'center_count_other'] = prev_fdh

            
        else:
            prev_total = state_df.loc[(state_fip,yr),'center_count_cc']
            prev_other = state_df.loc[(state_fip,yr),'center_count_fdh'] 
            prev_fdh = state_df.loc[(state_fip,yr),'center_count_other']
state_df.reset_index(inplace=True)
state_df.rename(columns=state_col_mapper,inplace=True)
state_df.to_csv('state.csv')


c:\learning\childCareDessert\childcare\scripts


In [78]:
state_df.head()

center_count_cc  center_count_fdh  center_count_other
stFips Year                                                       
1      2000              1.0               0.0                 0.0
       2001              1.0               0.0                 0.0
       2002              1.0               0.0                 0.0
       2003              1.0               1.0                 0.0
       2004              1.0               1.0                 0.0

In [89]:
import pandas as pd


## load raw data 
raw_data = pd.read_csv('../data/cc_cty_panel_.csv')
# raw_data_cols = raw_data.columns
raw_df = raw_data.fillna(0)


## mapping files
stfips = pd.read_csv('../data/state_fips.csv')
countyfips = pd.read_csv('../data/counties_fips.csv')

 

county_col_mapper = {
    'Year': 'Year',
    'center_count_cc': 'Total_CC',
    'center_count_fdh': 'FDH',
    'center_count_other': 'Other_CC',
    'state': 'State',
    'county_name':'County',
    'cty_fips':'cty_fips'
}

trend_col_mapper = {
    'Year': 'Year',
    'center_count_cc': 'Total_CC',
    'center_count_fdh': 'FDH',
    'center_count_other': 'Other_CC',
    'stFips': 'stFips',
    'pop04': 'pop'
}
# fips & name mapping
raw_data_full = pd.merge(left=raw_df,right=stfips,on=['stFips'])
raw_data_full = raw_data_full.merge(countyfips,on=['cty_fips','state'])
raw_data_full
# ## county map data
county_full_raw = raw_data_full[county_col_mapper.keys()] 
county_df = county_full_raw.rename(columns=county_col_mapper)
raw_data_full.groupby(by=['stFips','Year','state_name','state']).sum()
# ## trend data
# trend_data_grouped = raw_data_full.groupby(by=['stFips','year']).sum()[['pop04','capacity_total_cc','capacity_total_fdh','capacity_total_other']]

cty_fips   pop04  center_count_cc  \
stFips Year state_name state                                      
1      2000 Alabama    AL         1113    3519              1.0   
       2001 Alabama    AL         1113    3471              1.0   
       2002 Alabama    AL         1113    3537              1.0   
       2003 Alabama    AL         2186   46628              1.0   
       2004 Alabama    AL         2186   46477              1.0   
...                                ...     ...              ...   
55     2019 Wisconsin  WI      3965119  333184           3475.0   
       2020 Wisconsin  WI      3965119  331066           3382.0   
       2021 Wisconsin  WI      3965119  329324           3422.0   
       2022 Wisconsin  WI      3965119  321594           3394.0   
       2023 Wisconsin  WI      3965119  321594           3215.0   

                              capacity_total_cc  center_count_fdh  \
stFips Year state_name state                                        
1      2000 Alabama    AL                  54.0               0.0   
       2001 Alabama    AL                  54.0               0.0   
       2002 Alabama    AL                  54.0               0.0   
       2003 Alabama    AL                  54.0               1.0   
       2004 Alabama    AL                  54.0               1.0   
...                                         ...               ...   
55     2019 Wisconsin  WI              144373.0             774.0   
       2020 Wisconsin  WI              142533.0             656.0   
       2021 Wisconsin  WI              143400.0             655.0   
       2022 Wisconsin  WI              143100.0             645.0   
       2023 Wisconsin  WI              137220.0             557.0   

                              capacity_total_fdh  center_count_other  \
stFips Year state_name state                                           
1      2000 Alabama    AL                    0.0                 0.0   
       2001 Alabama    AL                    0.0                 0.0   
       2002 Alabama    AL                    0.0                 0.0   
       2003 Alabama    AL                    0.0                 0.0   
       2004 Alabama    AL                    0.0                 0.0   
...                                          ...                 ...   
55     2019 Wisconsin  WI                 2635.0                 0.0   
       2020 Wisconsin  WI                 2167.0                 0.0   
       2021 Wisconsin  WI                 2132.0                 0.0   
       2022 Wisconsin  WI                 2058.0                 0.0   
       2023 Wisconsin  WI                 1763.0                 0.0   

                              capacity_total_other  center_tot  \
stFips Year state_name state                                     
1      2000 Alabama    AL                      0.0           1   
       2001 Alabama    AL                      0.0           1   
       2002 Alabama    AL                      0.0           1   
       2003 Alabama    AL                      0.0           2   
       2004 Alabama    AL                      0.0           2   
...                                            ...         ...   
55     2019 Wisconsin  WI                      0.0        4249   
       2020 Wisconsin  WI                      0.0        4038   
       2021 Wisconsin  WI                      0.0        4077   
       2022 Wisconsin  WI                      0.0        4039   
       2023 Wisconsin  WI                      0.0        3772   

                              capacity_total  cc_desert  \
stFips Year state_name state                              
1      2000 Alabama    AL                 54        1.0   
       2001 Alabama    AL                 54        1.0   
       2002 Alabama    AL                 54        1.0   
       2003 Alabama    AL                 54        2.0   
       2004 Alabama    AL                 54        2.0   
...                                      ...       

In [79]:
raw_data_full = pd.merge(left=raw_df,right=stfips,on=['stFips'])
raw_data_full = raw_data_full.merge(countyfips,on=['cty_fips','state'])
raw_data_full.columns

Index(['stFips', 'cty_fips', 'Year', 'pop04', 'center_count_cc',
       'capacity_total_cc', 'center_count_fdh', 'capacity_total_fdh',
       'center_count_other', 'capacity_total_other', 'center_tot',
       'capacity_total', 'cc_desert', 'state_name', 'state', 'county_name'],
      dtype='object')

In [43]:
poprepopulate = ccdf.groupby(by=['state','year']).sum()[['pop04','center_count_cc','center_count_fdh','center_count_other']]
idxs = poprepopulate.index

mapfipcode = pd.read_csv('state_fips.csv')
inusefips = set(mapfipcode['id'])
in_data_state = set(poprepopulate.index.get_level_values(level='state'))
fips_lack = inusefips - in_data_state

for state in inusefips:
    prev_pop = 0    
    prev_total = 0
    prev_other = 0
    prev_fdh = 0
    for yr in range(2000,2024):
        pair = (state,yr)
        if pair not in idxs:
            poprepopulate.loc[(state,yr),'pop04'] = prev_pop
            poprepopulate.loc[(state,yr),'center_count_cc'] = prev_total
            poprepopulate.loc[(state,yr),'center_count_fdh'] = prev_other
            poprepopulate.loc[(state,yr),'center_count_other'] = prev_fdh
        else:
            prev_pop = poprepopulate.loc[(state,yr),'pop04']
            prev_total = poprepopulate.loc[(state,yr),'center_count_cc']
            prev_other = poprepopulate.loc[(state,yr),'center_count_fdh'] 
            prev_fdh = poprepopulate.loc[(state,yr),'center_count_other']

In [44]:
poprepopulate.reset_index().to_csv('final_trend2.csv')

In [40]:
popwithCode = pd.merge(mapfipcode, poprepopulate, left_on='id', right_on='state')[['name', 'code', 'state','year', 'pop04']]
pd.merge(popwithCode,df_all_years,left_on=['code','year'],right_on=['State','Year']).columns



Index(['name', 'code', 'state', 'year', 'pop04', 'State', 'Year', 'Other_CC',
       'Total_CC', 'FDH'],
      dtype='object')

In [42]:
pd.merge(popwithCode,df_all_years,left_on=['code','year'],right_on=['State','Year'])[['name', 'code', 'state', 'year', 'pop04', 'Other_CC','Total_CC', 'FDH']].to_csv('ChildcareCaseData.csv')

In [13]:
import pandas as pd

stateLevel = pd.read_csv('stateCaseData.csv')
alsoStateLevel = pd.read_csv('ChildcareCaseData.csv') # not using this file, but will need for population scale up line
countyLevel = pd.read_csv('countyCaseData.csv')




In [4]:
stateLevel.head()

,State,Year,Total_CC,ln_name,fipscode,Other_CC,FDC
0,AL,2019,3,Alabama,1,6,6
1,AL,2020,9,Alabama,1,3,9
2,AK,2019,6,ALASKA,2,4,6
3,AK,2020,6,ALASKA,2,6,9
4,AR,2019,9,Arkansas,5,7,6


In [7]:
countyLevel.head()

,fip,County,State,Year,Other_CC,Total_CC,FDH
0,3473,Baldwin,AL,2019,4,7,4
1,3493,Chilton,AL,2019,4,11,4
2,3558,Lauderdale,AL,2019,3,2,3
3,3625,Butler,AL,2019,3,71,3
4,3629,Tuscaloosa,AL,2019,3,250,3


In [21]:
presetgroup = countyLevel.groupby(by=['State','Year']).sum()[['Other_CC','Total_CC','FDH']].reset_index()
presetgroup.head()

,State,Year,Other_CC,Total_CC,FDH
0,AK,2019,60,58,60
1,AK,2020,41,60,41
2,AL,2019,437,19238,437
3,AL,2020,431,20185,431
4,AR,2019,407,991,407


In [28]:
presetgroup.merge(stateLevel[['State','Year','ln_name','fipscode']],on=['State','Year']).to_csv('statelevel2.csv')

In [27]:
stateLevel.columns

Index(['State', 'Year', 'Total_CC', 'ln_name', 'fipscode', 'Other_CC', 'FDC'], dtype='object')

In [6]:
import pandas as pd
df = pd.read_csv('cc_cty_panel_.csv')

In [8]:
trend = df.groupby(by=['state','year']).sum()['pop04']

In [11]:
trend = trend.reset_index()

In [23]:
cases = df.groupby(by=['state','year']).sum()[['capacity_total_cc','capacity_total_fdh','capacity_total_other']].fillna(0).reset_index()


In [31]:
trend_data = trend.merge(cases,on=['state','year'])
trend_data.head().columns

Index(['state', 'year', 'pop04', 'capacity_total_cc', 'capacity_total_fdh',
       'capacity_total_other'],
      dtype='object')

In [33]:
# map state name and fips
stateFips = pd.read_csv('state_fips.csv')
final_trend_dataset = pd.merge(left=trend_data,right=stateFips,left_on='state',right_on='id')[['state', 'year', 'pop04', 'capacity_total_cc', 'capacity_total_fdh', 'capacity_total_other','code']]

In [34]:
final_trend_dataset.to_csv('final_trend.csv')

In [4]:
import pandas as pd
pd.read_csv('final_trend.csv')

,fips,Year,pop2,Total_CC,FDH,Other_CC,State,pop
0,1,2000,3519,54,0,0,AL,3
1,1,2001,3471,54,0,0,AL,3
2,1,2002,3537,54,0,0,AL,3
3,1,2003,46628,54,0,0,AL,46
4,1,2004,46477,54,0,0,AL,46
...,...,...,...,...,...,...,...,...
772,55,2019,333184,144373,2635,0,WI,333
773,55,2020,331066,142533,2167,0,WI,331
774,55,2021,329324,143400,2132,0,WI,329
775,55,2022,321594,143100,2058,0,WI,321


## data required
1. final trend

['fips', 'Year', 'pop', 'Total_CC', 'FDH', 'Other_CC']

2. state case

['State', 'Year', 'ln_name', 'fipscode', 'Other_CC', 'FDH', 'Total_CC']

3. county case

['fip', 'County', 'State', 'Year', 'Other_CC', 'Total_CC', 'FDH']


## original data
1. cc_cty_panel_.csv

['state', 'cty_fips', 'year', 'pop04', 'center_count_cc',
       'capacity_total_cc', 'center_count_fdh', 'capacity_total_fdh',
       'center_count_other', 'capacity_total_other', 'center_tot',
       'capacity_total', 'cc_desert']

In [10]:
import pandas as pd
final_trend_cols = pd.read_csv('../data/final_trend2.csv').columns
state_case_cols = pd.read_csv('../data/stateCaseData.csv').columns
county_case_cols = pd.read_csv('../data/countyCaseData.csv').columns
raw_data = pd.read_csv('../data/cc_cty_panel_.csv').columns
raw_data

Index(['state', 'cty_fips', 'year', 'pop04', 'center_count_cc',
       'capacity_total_cc', 'center_count_fdh', 'capacity_total_fdh',
       'center_count_other', 'capacity_total_other', 'center_tot',
       'capacity_total', 'cc_desert'],
      dtype='object')

### county data preparation

In [40]:
raw_df = pd.read_csv('../data/cc_cty_panel_.csv')
stfips = pd.read_csv('../data/state_fips.csv')
countyfips = pd.read_csv('../data/counties_fips.csv')
raw_df = raw_df.fillna(0)
countydf = raw_df[['cty_fips','state','year','center_count_cc','center_count_fdh','center_count_other']]

countydf1 = pd.merge(left=countydf,right=stfips[['id','code']],left_on='state',right_on='id')
countydf2 = countydf1[['cty_fips','year','center_count_cc','center_count_fdh','center_count_other','code']]
countydf3 = pd.merge(left=countydf2, right=countyfips[['fips','state','name']],left_on='cty_fips',right_on='fips')


countydf3 = countydf3.rename(columns={
    'year': 'Year',
    'center_count_cc': 'Total_CC',
    'center_count_fdh': 'FDH',
    'center_count_other': 'Other_CC',
    'code': 'State',
    'name':'County',
    'fips': 'fips'
})


,Year,Total_CC,FDH,Other_CC,State,fips,County
0,2003,0.0,1.0,0.0,AL,1073,Jefferson
1,2004,0.0,1.0,0.0,AL,1073,Jefferson
2,2005,0.0,1.0,0.0,AL,1073,Jefferson
3,2005,0.0,1.0,0.0,AL,1101,Montgomery
4,2006,0.0,1.0,0.0,AL,1101,Montgomery
...,...,...,...,...,...,...,...
30991,2019,39.0,25.0,0.0,WI,55141,Wood
30992,2020,34.0,20.0,0.0,WI,55141,Wood
30993,2021,34.0,20.0,0.0,WI,55141,Wood
30994,2022,29.0,15.0,0.0,WI,55141,Wood


In [43]:
countydf3.to_csv('final_county_case.csv',index=False)